In [4]:
# !pip install langgraph
# !pip install aiosqlite 
# !pip install langchian

In [10]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

In [11]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(dict)

In [12]:
class State(TypedDict):
    name: str
    age: int
    messages: Annotated[list[AnyMessage], add_messages]


In [13]:
from langchain_core.runnables import RunnableConfig

def generateRandomName(state: dict, config: RunnableConfig) -> State:
    print(config)
    return { "name": "Vishal", "age": 27}

In [14]:
def actionNode(state: dict) -> State:
    return state

In [15]:
builder.add_node("entry", generateRandomName)
builder.add_node("action", actionNode)

In [16]:
builder.add_edge(START, "entry")
builder.add_edge("entry", "action")
builder.add_edge("action", END)

In [21]:
import asyncio
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver
from langgraph.checkpoint.sqlite import SqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
memory = SqliteSaver.from_conn_string(":memory:")

In [22]:
graph = builder.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "1"}}

inputs = {
    "name": "Ram",
    "age": 12
}

In [25]:
graph.invoke(input=inputs, config=config)

{'tags': [], 'metadata': {'thread_id': '1', 'langgraph_step': 1, 'langgraph_node': 'entry', 'langgraph_triggers': ['start:entry'], 'langgraph_task_idx': 0, 'thread_ts': '1ef51cfb-f940-62ec-8000-6ad5a2ccf43d'}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x10c851040>, 'configurable': {'thread_id': '1-entry', '__pregel_send': functools.partial(<function local_write at 0x10c6e18a0>, <built-in method extend of collections.deque object at 0x129717100>, {'__start__': PregelNode(config={'tags': ['langsmith:hidden'], 'metadata': {}, 'configurable': {}}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<__root__>(recurse=True, writes=[ChannelWriteEntry(channel='__root__', value=<object object at 0x105f16160>, skip_none=True, mapper=None)], require_at_least_one_of=['__root__']), ChannelWrite<start:entry>(recurse=True, writes=[ChannelWriteEntry(channel='start:entry', value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)])

{'name': 'Vishal', 'age': 27}